# Comparison of Sentinel-1 RTC products from different software

In [ ]:
import os
import fsspec
import h5py
import numpy as np
import matplotlib.pyplot as plt
import requests
import boto3
import pandas as pd
import json

%matplotlib inline

# Settings

In [ ]:
# general structure for scenes in s3
# s3_bucket/software/dem/scene/scene_files
s3_bucket = 'deant-data-public-dev'
softwares = ['pyrosar','rtc-opera']
scenes = [
    'S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63',
    'S1A_IW_SLC__1SSH_20190605T222724_20190605T222751_027550_031BE1_AD3A'
]
dem = 'glo_30'

# get credebtials for AWS
with open('aws_credentials.txt') as f:
    ACCESS_ID, ACCESS_KEY = f.readlines()
    ACCESS_ID = ACCESS_ID.strip()
    ACCESS_KEY = ACCESS_KEY.strip()

# setup s3
s3 = boto3.client('s3', 
                        region_name='ap-southeast-2',
                        aws_access_key_id=ACCESS_ID,
                        aws_secret_access_key= ACCESS_KEY)

## Show example scene files for software

In [ ]:
for software in softwares:
    print(f'software : {software}')
    params = {
        "Bucket": f'{s3_bucket}',
        "Prefix": f'{software}/{dem}/{scenes[0]}'
    }
    objects = s3.list_objects_v2(**params)
    if 'Contents' in objects.keys():
        for x in objects['Contents']:
            print(x)
    print('\n')

# Compare timing

In [ ]:
timing_data = []
for software in softwares:
    for scene in scenes:
        timing_file = f'{software}/{dem}/{scene}/{scene}_timing.json'
        try:
            s3.download_file(s3_bucket, timing_file, 'tmp.json')
            with open('tmp.json') as json_file:
                data = json.load(json_file)
                data['software'] = software
                data['scene'] = scene
            timing_data.append(data)
            print(f'downloaded: {timing_file}')
        except:
            print(f'no timing file: {timing_file}')

df_timing = pd.DataFrame.from_records(timing_data, index=['software','scene'])
df_timing

In [ ]:
import fsspec
import h5py
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
f = '/data/OPERA/outdir/OPERA_L2_RTC-S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63_REMA500.h5'
fs1 = fsspec.open(f, mode='rb', anon=True, default_fill_cache=False)
f1 = h5py.File(fs1.open(),'r')

In [ ]:
list(f1.keys())

In [ ]:
f1['metadata']['processingInformation']['parameters'].keys()

In [ ]:
for x in f1['data'].keys():
    print(x + ' : ', f1['data'][x])

In [ ]:
np.nanmax(f1['data']['numberOfLooks'][:])